In [ ]:
!pip install 'tensorflow[and-cuda]==2.9.0' ampligraph==2.0.1
!pip install pipe
!pip install gdown

In [ ]:
!gdown 1qO0gj4RTY0ErNb8VXyaSeF4p8Y1jRPHb

In [1]:
import numpy as np
import pandas as pd
import ampligraph
import tensorflow as tf

tf.test.is_gpu_available()

2023-12-12 16:02:59.630353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 16:02:59.630391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 16:02:59.632291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 16:02:59.639980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 16:03:00.368350: W tensorflow/compiler/tf2

True

2023-12-12 16:03:01.093378: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-12 16:03:01.093616: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-12 16:03:01.152505: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
from pipe import select, where

In [3]:
from rdflib import Graph, URIRef
graph = Graph()
graph.parse('res.ttl', format='ttl')

<Graph identifier=N62c2499a13ae4a0fad1228235e913d64 (<class 'rdflib.graph.Graph'>)>

In [4]:
def is_not_literal(triplet):
    return isinstance(triplet[0], URIRef) and isinstance(triplet[2], URIRef)

def is_not_rdf_ns(triplet, namespaces_to_skip=("http://www.w3.org/",)):
    predicate_uri = str(triplet[1])
    if any(predicate_uri.startswith(namespace) for namespace in namespaces_to_skip):
        return False
    return True

def normalize(triplet):
    sub, pre, obj = triplet
    
    subject_fragment = sub.split("/")[-1]
    object_fragment = obj.split("/")[-1]
    predicate_fragment = pre.split("#")[-1] if "#" in str(pre) else pre.split("/")[-1]

    return subject_fragment, predicate_fragment, object_fragment

In [5]:
triplets = list(graph.triples((None, None, None)) 
               | where(is_not_literal)
               | where(is_not_rdf_ns)
               | select(normalize))

In [6]:
triplets[1]

('70892', 'isWinner', 'Shibano_Ryunosuke')

In [7]:
df = pd.DataFrame(triplets, columns=['subject', 'predicate', 'object'])

In [8]:
df.predicate.unique()

array(['isLoser', 'isWinner', 'representsTheCountry',
       'playedInTournament', 'isChampion', 'isRunnerup', 'inTheCountry'],
      dtype=object)

In [9]:
df[(df.subject=="88506") | (df.object=="88506")]

,subject,predicate,object
64215,88506,isWinner,Lee_Jihyun_(m)
100792,88506,isLoser,Yun_Junsang


In [10]:
from ampligraph.evaluation import train_test_split_no_unseen

In [11]:
X_train, X_valid = train_test_split_no_unseen(np.array(triplets), test_size=10000)

In [12]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

Train set size:  (223386, 3)
Test set size:  (10000, 3)


In [13]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer

# Initialize a ComplEx neural embedding model
model = ScoringBasedEmbeddingModel(k=128,
                                   eta=30,
                                   scoring_type='ComplEx')


# Optimizer, loss and regularizer definition
optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = get_loss('multiclass_nll')
regularizer = get_regularizer('LP', {'p': 3, 'lambda': 1e-5})

# Compilation of the model
model.compile(optimizer=optim,
              loss=loss,
              entity_relation_regularizer=regularizer)

2023-12-12 16:03:30.932044: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-12 16:03:30.932281: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-12 16:03:30.932465: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [14]:
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)


# Fit the model
model.fit(X_train,
          batch_size=int(X_train.shape[0] / 50),
          epochs=100,  # Number of training epochs
          verbose=True  # Enable stdout messages
          )

Epoch 1/100


2023-12-12 16:03:45.700444: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-12 16:03:47.292708: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fdf14494650 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-12 16:03:47.292735: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-12-12 16:03:47.297401: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-12 16:03:47.328914: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1702386227.401497   28461 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


52/52 [==============================] - 5s 94ms/step - loss: 15041.2705
Epoch 2/100
 4/52 [=>............................] - ETA: 1s - loss: 15057.8301

2023-12-12 16:03:49.388379: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17389022229631790751
2023-12-12 16:03:49.388407: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17404393642466714328


52/52 [==============================] - 2s 37ms/step - loss: 15041.1240
Epoch 3/100
 4/52 [=>............................] - ETA: 1s - loss: 15049.6309

...

2023-12-12 16:07:01.918800: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17389022229631790751
2023-12-12 16:07:01.918846: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17404393642466714328


52/52 [==============================] - 2s 39ms/step - loss: 7685.5771
Epoch 100/100
 4/52 [=>............................] - ETA: 1s - loss: 7681.5161

2023-12-12 16:07:03.955608: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17389022229631790751
2023-12-12 16:07:03.955639: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17404393642466714328


52/52 [==============================] - 2s 39ms/step - loss: 7617.0518


2023-12-12 16:07:05.994181: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17389022229631790751
2023-12-12 16:07:05.994209: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17404393642466714328


In [ ]:
list(df.itertuples(index=False, name=None))

In [17]:
from ampligraph.utils import save_model, restore_model

save_model(model, model_name_path='goemb.pkl')

In [18]:
model = restore_model(model_name_path='goemb.pkl')

Saved model does not include a db file. Skipping.


In [19]:
model

In [20]:
X_train

array([['39018', 'isLoser', 'Nishida_Terumi'],
       ['75385', 'isWinner', 'Fan_Yin'],
       ['68344', 'isWinner', 'Lin_Junyan'],
       ...,
       ['75643', 'isWinner', 'Kang_Hun'],
       ['7637', 'isLoser', 'Rui_Naiwei'],
       ['90344', 'isWinner', 'Kim_Yeonghwan']], dtype='<U49')

In [21]:
X_valid

array([['19362', 'isLoser', 'Dang_Yifei'],
       ['90982', 'isWinner', 'Lin_Lixiang'],
       ['40723', 'isLoser', 'Yi_Lingtao'],
       ...,
       ['109813', 'isLoser', 'Li_Weiqing'],
       ['91006', 'isWinner', 'Lu_Yuhua'],
       ['8th_Korean_LG_Cup', 'isChampion', 'Lee_Changho']], dtype='<U49')

In [22]:
filter_triples = {'test': np.concatenate((X_train, X_valid))}

In [23]:
ranks = model.evaluate(X_valid,
                      #  model=model,
                       use_filter=filter_triples,
                      #  use_default_protocol=True,
                       verbose=True)

314/314 [==============================] - 89s 283ms/step


2023-12-12 16:14:33.808639: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 363017895887990967
2023-12-12 16:14:33.808669: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5159017537505981313
2023-12-12 16:14:33.808684: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15734846863950841227
2023-12-12 16:14:33.808695: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4823147363026306933
2023-12-12 16:14:33.808705: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2752503148928082198
2023-12-12 16:14:33.808734: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8418342793312690848
2023-12-12 16:14:33.808746: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv ite

In [24]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10", (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3", hits_3)
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1", hits_1)

MRR: 0.03
MR: 41115.73
Hits@10 0.04585
Hits@3 0.0275
Hits@1 0.01785


# Links prediction

In [25]:
df = pd.DataFrame(X_train,columns = ['subject','predicate','object'])
matchSubject = "88506"
print(df[df.subject==matchSubject])

       subject predicate          object
133751   88506   isLoser     Yun_Junsang
135774   88506  isWinner  Lee_Jihyun_(m)


In [26]:
df['object'].iloc[:10]

0      Nishida_Terumi
1             Fan_Yin
2          Lin_Junyan
3       Park_Yeonghun
4         Cho_Hunhyun
5       Huang_Yizhong
6     Nakamura_Shinya
7       Onishi_Ryuhei
8    Shibano_Toramaru
9       Fang_Tianfeng
Name: object, dtype: object

In [30]:
dfFiltered = np.array(
    df[
        (df.subject != matchSubject)
        | ((df.subject == matchSubject) & ~df.predicate.isin(['isLoser', 'isWinner']))
    ]
)

In [31]:
statements = np.array([
    [f'{matchSubject}', 'isWinner', 'Lee_Jihyun_(m)'],
    [f'{matchSubject}', 'isWinner', 'Yun_Junsang'],
    [f'{matchSubject}', 'isLoser', 'Lee_Jihyun_(m)'],
    [f'{matchSubject}', 'isLoser', 'Yun_Junsang'],
    [f'{matchSubject}', 'isWinner', 'Nishida_Terumi'],
    [f'{matchSubject}', 'isWinner', 'Lin_Junyan'],
])

In [32]:
statements_filter = {'test': np.array(list({tuple(i) for i in np.vstack((dfFiltered, statements))}))}
statements_filter

{'test': array([['65298', 'isLoser', 'So_Yokoku'],
        ['80941', 'isLoser', 'Uchida_Shuhei'],
        ['86400', 'isWinner', 'Zheng_Xu'],
        ...,
        ['49758', 'isLoser', 'Kim_Hyemin'],
        ['1178', 'isLoser', 'Yoda_Norimoto'],
        ['25509', 'playedInTournament', '15th_LG_Cup']], dtype='<U49')}

In [33]:
ranks_statements = model.evaluate(dfFiltered,
                      #  model=model,
                        use_filter=statements_filter,
                      #  use_default_protocol=True,
                       corrupt_side = 's+o',
                        verbose=True)

6982/6982 [==============================] - 1927s 276ms/step


2023-12-12 16:59:22.397518: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6727112693315944653
2023-12-12 16:59:22.397561: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15853317699967754649
2023-12-12 16:59:22.397574: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14661881145944039943
2023-12-12 16:59:22.397585: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5558320293950874615
2023-12-12 16:59:22.397595: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10480725463224738745
2023-12-12 16:59:22.397605: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4197363760740976369
2023-12-12 16:59:22.397615: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

In [35]:
!pip install joblib

In [36]:
import joblib

joblib.dump(ranks_statements, "ranks_statements.joblib")

['ranks_statements.joblib']

In [34]:
scores = model.predict(statements)
scores

2023-12-12 17:18:48.102539: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5339189334493540054


array([ 6.107538  ,  0.38355687,  0.8535739 ,  6.422471  ,  0.04226655,
       -0.7592815 ], dtype=float32)

In [38]:
from scipy.special import expit
probs = expit(scores)

pd.DataFrame(list(zip([' '.join(x) for x in statements],
                      ranks_statements,
                      np.squeeze(scores),
                      np.squeeze(probs))),
             columns=['statement', 'rank', 'score', 'prob']).sort_values("prob")

,statement,rank,score,prob
5,88506 isWinner Lin_Junyan,[9],-0.759282,0.318802
4,88506 isWinner Nishida_Terumi,[16],0.042267,0.510565
1,88506 isWinner Yun_Junsang,[14],0.383557,0.594731
2,88506 isLoser Lee_Jihyun_(m),[7],0.853574,0.701316
0,88506 isWinner Lee_Jihyun_(m),[5],6.107538,0.997779
3,88506 isLoser Yun_Junsang,[12],6.422471,0.998378
